## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import pandas as pd
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint, ModelSummary

import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.two_tower.model import TwoTowerRating
from src.algo.two_tower.dataset import UserItemRatingDFDataset, UserItemBinaryDFDataset
from src.algo.two_tower.trainer import TwoTowerLitModule

In [3]:
load_dotenv(override = True)

True

In [4]:
hidden_dim = 128
embedding_dim: int = 128

In [5]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "first-attempt"
    run_name: str = f"005-two-tower-{embedding_dim}-{hidden_dim}-neg-4"
    notebook_persit_dp: str = None
    
    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "two-tower"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128
    embedding_dim: int = embedding_dim
    learning_rate: float = 0.01
    l2_reg: float = 1e-5
    early_stopping_patience: int = 10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    max_epochs: int = 100

    # TwoTower specific
    hidden_dim: int = hidden_dim
    dropout: float = 0
    
    best_checkpoint_path: str = None

    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_neg_4.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_neg.parquet")

    def init(self):
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-05-13 16:09:22.356 | INFO     | __main__:init:42 - Setting up Mlflow experiment: first-attempt, run_name: 005-two-tower-128-128-neg-4


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "first-attempt",
  "run_name": "005-two-tower-128-128-neg-4",
  "notebook_persit_dp": "/home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/005-two-tower-128-128-neg-4",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "two-tower",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128,
  "embedding_dim": 128,
  "learning_rate": 0.01,
  "l2_reg": 0.00001,
  "early_stopping_patience": 10,
  "device": "cuda",
  "max_epochs": 100,
  "hidden_dim": 128,
  "dropout": 0.0,
  "best_checkpoint_path": null,
  "train_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/train_sample_interactions_16407u_neg_4.parquet",
  "val_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/val_sample_interactions_16407u_neg.parquet"
}


## Init model

In [6]:
def init_model(n_user, n_items, embedding_dim, hidden_dim, dropout):
    """
    Initialize the model with the given parameters.
    """
    model = TwoTowerRating(
        num_users = n_user,
        num_items = n_items,
        embedding_dim = embedding_dim,
        hidden_units_dim = hidden_dim,
        dropout = dropout,
    )
    return model

## Test implementation

In [7]:
embedding_dim = 8
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 1, 0, 1, 0]

n_users = len(set(user_indices))
n_items = len(set(item_indices))

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
    }
)

model = init_model(n_users, n_items, embedding_dim, embedding_dim, args.dropout)

# Example forward pass
model.eval()
user = torch.tensor([0])
target_item = torch.tensor([2])
predictions = model(user, target_item)
print(predictions)
model.train()

tensor([-0.1569], grad_fn=<SumBackward1>)


TwoTowerRating(
  (item_embedding): Embedding(5, 8)
  (item_fc): Linear(in_features=8, out_features=8, bias=True)
  (user_embedding): Embedding(3, 8)
  (user_fc): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=False)
  (sigmoid_fn): Sigmoid()
)

In [8]:
rating_dataset = UserItemRatingDFDataset(
    train_df,
    user_col = "user_indice",
    item_col = "item_indice",
    rating_col = args.rating_col,
    timestamp_col = args.timestamp_col,
)
train_loader = DataLoader(
    rating_dataset,
    batch_size = batch_size,
    shuffle = False,
)

In [9]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.])}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([0., 1.])}
{'user': tensor([2]), 'item': tensor([4]), 'rating': tensor([0.])}


In [10]:
# model
lit_model = TwoTowerLitModule(model, log_dir=args.notebook_persit_dp)

# callbacks
callbacks = [ModelSummary(max_depth=-1)]
# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persit_dp}/logs/test",
    max_epochs=300,
    accelerator=args.device if args.device else "auto",
    callbacks=callbacks
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

2025-05-11 18:35:16.466 | INFO     | src.algo.two_tower.trainer:__init__:59 - IDM is not provided. Skipping Evidently ranking metrics logging.
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.


In [7]:
users =torch.tensor([1, 1, 1, 1])
items = torch.tensor([0, 1, 2, 3])

predictions = model(users, items)
print(predictions)

NameError: name 'model' is not defined

## Trainig loop

In [18]:
train_df = pd.read_parquet(args.train_data_fp)
val_df = pd.read_parquet(args.val_data_fp)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [19]:
train_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
150650,AEZIRQUUFY4STOKOPL73DSJTDDOA,B0C3GSXJDJ,0.0,2012-11-21 01:21:44.000,4081,3326,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
337369,AFVUGFKE272XDSYJMORYZWGWQEZA,B0BPN2DXC1,0.0,2016-01-28 07:04:08.000,7636,3500,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
635293,AHI7UQNLAOXUWS6O6VCPKHP5HVGA,B00AA9Q94C,0.0,2020-10-28 01:35:34.712,14101,3793,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 535.0, 14..."


## Convert user_id and item_id into indices

In [8]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

'AE227WAM4NWQPJI33OPN7ZARNNZQ'

In [15]:
train_df = train_df.pipe(idm.map_indices)
val_df = val_df.pipe(idm.map_indices)

assert idm.unknown_item_index not in train_df["item_indice"].values, "Unknown item index must be present in training data."
assert idm.unknown_user_index not in train_df["user_indice"].values, "Unknown user index must be present in training data."
assert idm.unknown_item_index not in val_df["item_indice"].values, "Unknown item index must be present in validation data."
assert idm.unknown_user_index not in val_df["user_indice"].values, "Unknown user index must be present in validation data."

In [16]:
train_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
150650,AEZIRQUUFY4STOKOPL73DSJTDDOA,B0C3GSXJDJ,0.0,2012-11-21 01:21:44.000,4081,4734,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
337369,AFVUGFKE272XDSYJMORYZWGWQEZA,B0BPN2DXC1,0.0,2016-01-28 07:04:08.000,7636,4592,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
635293,AHI7UQNLAOXUWS6O6VCPKHP5HVGA,B00AA9Q94C,0.0,2020-10-28 01:35:34.712,14101,1083,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 535.0, 14..."


In [20]:
assert train_df.groupby(args.user_col)[args.item_col].nunique().min() >= 5, "Each user must have at least five items."
assert train_df.groupby(args.item_col)[args.user_col].nunique().min() >= 10, "Each item must have at least ten users."

In [21]:
train_df = train_df.sample(frac=1, random_state=42)

In [25]:
rating_dataset = UserItemBinaryDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)
val_rating_dataset = UserItemBinaryDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [20]:
val_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice
260331,AGQFM7GX5UGRCK5F6EEGEEB25FKQ,B01C4W2P18,0.0,2021-06-02 04:46:55.478,10995,2482
259198,AGYITA5HB3G7B5UQIIYBVCPLRFVA,B07GXDLJP9,0.0,2022-01-20 17:33:34.286,12058,3441
258841,AFKBE4VLE3XEQ5IHUZI2Q5KAKFCQ,B00XAJD7NA,0.0,2021-08-15 20:42:07.057,6169,2147
259382,AHKE5QFJM747GCEVBUF7QRH47HJA,B08RLW7918,0.0,2021-07-16 00:07:10.093,14395,4018
127982,AGLAJCRBNNNKVIASX3KKSULI2CFQ,B006R0VWSG,4.0,2022-01-21 18:46:08.771,10330,769
...,...,...,...,...,...,...
130708,AGPDIY2JHE7EVMJTAV3SCLYDORQA,B0723D3FVL,4.0,2021-05-26 16:44:51.699,10867,2986
261331,AHW4A2IHDBOQ3RAKPRE34TGEQRSA,B0BS2ZMHCL,0.0,2021-10-01 09:04:57.543,15908,4616
130349,AFQTOVETKDBVDIHKIPEZMPMMLCMQ,B0C2HWSXNL,5.0,2021-01-14 13:43:17.216,7022,4718
130673,AEZKOL32LQNPV2K5R3U25Q3GGQQA,B07WQKKS8V,5.0,2021-06-25 21:29:34.271,4085,3748


In [21]:
for i in train_loader:
    print(i)
    break

{'user': tensor([ 7112, 12526,  1408,  2539,  8510,  8647,  8116,  3793,  9475,  4357,
         8053,  3073,   279,  9600, 11221,  5002,  5106,  3262,  6671,   368,
         5089, 15438,  9382,  3230,  7946,  9648,  4658, 16199, 14604,  1439,
         9048,  9005, 15427,  3258,  9106,  4537,  4199,  6625,  5578,  6169,
         6037,  4245, 14237, 11629,  7101,  8482,  9886,  4018, 12834,  2325,
        14524, 16358,  5240, 14751,  7079,  4831,   214,  1382,  1897,  1710,
         3952,  3186,  6193,  3226,  4818,  2395, 15038,    51,  3220, 11986,
        11062, 12612,  1605, 13328,  2596, 12191,  9949,  5294,   285,  6944,
         2381,  3842, 15607,  9296,  4341,  6784,  5772, 13780,  9930, 13750,
        13152, 10928, 10286,  4896,  5563,  3894,  8523,  1177,  6820,  6255,
        14523, 15589, 10565,  2687, 14440,  1540,  9555, 14018,  7676, 12177,
        15337,  8669, 12012,  6019, 16098,  8001, 12590, 14299, 13322,   910,
        15865, 14963, 15468,  2595,  5447, 11392,  6996

In [22]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")
model = init_model(n_users, n_items, args.embedding_dim, args.hidden_dim, args.dropout)

2025-05-13 16:19:25.382 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817


## Overfir 1 batch

In [23]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model_summary = ModelSummary(max_depth=-1)

model = init_model(n_users, n_items, args.embedding_dim, args.hidden_dim, args.dropout)
lit_model = TwoTowerLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm,
    log_ranking_metrics=False
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping, model_summary],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type                   | Params | Mode 
------------------------------------------------------------------------
0 | model                | TwoTowerRating         | 2.7 M  | train
1 | model.item_embedding | Embedding              | 616 K  | train
2 | model.item_fc        | Linear                 | 16.5 K | train
3 | model.user_embedding | Embedding              | 2.1 M  | train
4 | model.user_fc        | Linear                 | 16

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/conne

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
2025-05-11 18:36:04.493 | INFO     | __main__:<module>:33 - Logs available at /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/005-two-tower-128-128-neg-4/logs/overfit/lightning_logs/version_3


## Run on all data

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_roc_auc", patience=5, mode="max", verbose=False, min_delta=0.01
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persit_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_roc_auc",
    mode="max",
)

model_summary = ModelSummary(max_depth=-1)

model = init_model(n_users, n_items, args.embedding_dim, args.hidden_dim, args.dropout)
lit_model = TwoTowerLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
    
)

log_dir = f"{args.notebook_persit_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback, model_summary],
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

# Change the library as a workaround for the issue in the latest Lightning release
#https://github.com/Lightning-AI/pytorch-lightning/pull/20669/commits/429f732a0528c558e701da7ec01e51c1e2e4f32e

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/005-two-tower-128-128-neg-4/checkpoints exists and is not empty.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type                   | Params | Mode 
------------------------------------------------------------------------
0 | model                | TwoTowerRating         | 2.7 M  | train
1 | model.item_embedding | Embedding              | 616 K  | train
2 | model.item_fc        | Linear                 | 16.5 K | train
3 | model.user_embedding | Em

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Clean up

In [ ]:
all_params = [args]

if args.log_to_mlflow:
    run_id = trainer.logger.run_id

    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.model_dump()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

🏃 View run 005-two-tower-128-128-neg-4 at: http://138.2.61.6:5002/#/experiments/2/runs/fd51bfab00c54c5290ce3691d6aafbbd
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


## Log metrics

In [ ]:
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics

In [ ]:
model

TwoTowerRating(
  (item_embedding): Embedding(4817, 128)
  (item_fc): Linear(in_features=128, out_features=128, bias=True)
  (user_embedding): Embedding(16407, 128)
  (user_fc): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=False)
  (sigmoid_fn): Sigmoid()
)

In [ ]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = TwoTowerLitModule.load_from_checkpoint(
    checkpoint_path=checkpoint_callback.best_model_path,
    model = init_model(n_users, n_items, 128,128, args.dropout),
)

2025-05-11 18:31:02.366 | INFO     | __main__:<module>:1 - Loading best checkpoint from /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/005-two-tower-128-128-neg-4/checkpoints/best-checkpoint-v2.ckpt...
2025-05-11 18:31:02.736 | INFO     | src.algo.two_tower.trainer:__init__:59 - IDM is not provided. Skipping Evidently ranking metrics logging.


In [ ]:
best_model = best_trainer.model.to(args.device)
best_model.eval()

TwoTowerRating(
  (item_embedding): Embedding(4817, 128)
  (item_fc): Linear(in_features=128, out_features=128, bias=True)
  (user_embedding): Embedding(16407, 128)
  (user_fc): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=False)
  (sigmoid_fn): Sigmoid()
)

In [ ]:
val_recs_df = val_df.sort_values(by=args.timestamp_col).drop_duplicates(subset=[args.user_col], keep="first")

In [ ]:
mlflow.start_run(run_id = trainer.logger.run_id)

<ActiveRun: >

## Classification metrics

In [ ]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values

In [ ]:
len(val_user_indices), len(val_item_indices)

(6958, 6958)

In [ ]:
users = torch.tensor(val_user_indices, device=args.device)
items = torch.tensor(val_item_indices, device=args.device)
classifications = best_model.predict(users, items)

In [ ]:
eval_classification_df = val_df.assign(
    classification_proba=classifications.cpu().detach().numpy(),
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)

In [ ]:
eval_classification_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,classification_proba,label
260331,AGQFM7GX5UGRCK5F6EEGEEB25FKQ,B01C4W2P18,0.0,2021-06-02 04:46:55.478,10995,2482,0.505740,0
259198,AGYITA5HB3G7B5UQIIYBVCPLRFVA,B07GXDLJP9,0.0,2022-01-20 17:33:34.286,12058,3441,0.533352,0
258841,AFKBE4VLE3XEQ5IHUZI2Q5KAKFCQ,B00XAJD7NA,0.0,2021-08-15 20:42:07.057,6169,2147,0.515961,0


In [ ]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

## Ranking metrics

In [ ]:
val_recs_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice
129191,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B004FV4ROA,1.0,2020-12-27 00:30:31.146,11295,528
128040,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B08F1P3BCC,2.0,2020-12-27 01:44:52.242,1784,3925
128167,AGAVHCK42EGMVS7DGPRX6HBCUCNQ,B09Q3NR84W,5.0,2020-12-27 02:25:48.357,9042,4273
127542,AEFVBMCJAFNULDI5V2CKKTBCPURA,B07N1L5HX1,5.0,2020-12-27 02:32:15.171,1542,3550
258856,AGLXMKHBLTBNT3X2CLBAPW6QUTQA,B001EAQTRI,0.0,2020-12-27 03:37:22.772,10418,212
...,...,...,...,...,...,...
258901,AGGDNWGN3NDJ2DI5CBSFOMUAM6XA,B076XFGK32,0.0,2022-02-18 19:43:25.492,9711,3115
258608,AEKUF6AOVWDWFYOKPWO2CV72PEDQ,B009OBCAW2,0.0,2022-02-19 01:32:51.519,2171,1042
129601,AFBTD25HPE4BE4LUFV3DTI2E2N2A,B07TMJ8S5Z,5.0,2022-02-19 16:49:57.966,5159,3699
130454,AHLN6GKTKZE22AON34YAQXTGK63A,B0C682GZ5X,5.0,2022-02-19 17:28:55.519,14550,4772


In [ ]:
recommendations = best_model.recommend(
    torch.tensor(val_recs_df["user_indice"].values, device=args.device),
    top_k=args.top_K,
    batch_size=1)

Generating recommendations:   0%|          | 0/2424 [00:00<?, ?it/s]

In [ ]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,11295,2962,0.632960,1.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B071JN4FW6
1,11295,4172,0.622946,2.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B099V8GPR4
2,11295,4659,0.608351,3.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B0BW4PFM58
3,11295,4327,0.607935,4.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B09WLDFF2T
4,11295,2616,0.604010,5.0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B01H355BGM
...,...,...,...,...,...,...
242395,2446,337,0.564083,96.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B002QUZM0U
242396,2446,2777,0.563983,97.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B01M7NFWQ5
242397,2446,4111,0.563936,98.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B0939S6BWT
242398,2446,3770,0.563785,99.0,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B07Y59K6NG


In [ ]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
129956,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B091K4WYD1,4.0,1.0
127398,AHZ6GFHFM6Z7CRPSXRIYQ5Z7GERQ,B07JMQP6T6,5.0,1.0
130809,AFQZQHAMZHP54BLVW3AZG2NDKAQA,B01N27P7ME,3.0,1.0
129383,AH7L2ZE36P7Q7ZDTDE2FIWWBU7ZA,B0B5J7MLTS,5.0,1.0
128262,AGOAZS3ZJNV74POYA7OW2JBZYAQQ,B0B2Y5WYRG,5.0,1.0
...,...,...,...,...
259295,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B0BSF17PM2,0.0,17.0
259294,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B003XRES32,0.0,18.0
258813,AEN2KQVSR5TWRXNQS3OTFT4EZQCA,B07D4Z36V8,0.0,18.0
259293,AFKERAMSXU4MWO3H53R7DEFOHUVQ,B0051VVOB2,0.0,19.0


In [ ]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
77,8.0,4073.0,0.611707,1,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B08ZN7YXTQ,0,NaN
1,8.0,316.0,0.599588,2,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B002J9HBIO,0,NaN
24,8.0,2011.0,0.597690,3,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B00UAKWFB2,0,NaN
44,8.0,3090.0,0.596229,4,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B075XMZMWY,0,NaN
90,8.0,4411.0,0.594314,5,AE24AB4DW5KYK3F5DYOT5VPW2VLA,B0B4QS4F25,0,NaN
...,...,...,...,...,...,...,...,...
248905,16403.0,1286.0,0.566312,98,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B00DN3J03O,0,NaN
248942,16403.0,3131.0,0.566249,99,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B077D5MC5M,0,NaN
248976,16403.0,3768.0,0.566094,100,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B07Y4822BG,0,NaN
248938,NaN,NaN,NaN,101,AHZZM7BCJAF2UEMMBHZCLXBB2SVA,B075QC3TZY,1,1.0


In [ ]:
ranking_report = log_ranking_metrics(args, eval_df)

In [ ]:
mlflow.end_run()

🏃 View run 005-two-tower-128-128-neg-4 at: http://138.2.61.6:5002/#/experiments/2/runs/fd51bfab00c54c5290ce3691d6aafbbd
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


## Register model

In [10]:
run_id = "fd51bfab00c54c5290ce3691d6aafbbd"
mlf_client = mlflow.tracking.MlflowClient()

In [11]:
# Persit idm to mlflow
mlf_client.log_artifact(
    run_id=run_id,
    local_path=idm_path,
)

In [130]:
from src.algo.two_tower.inference import TwoTowerInferenceWrapper
from mlflow.models.signature import infer_signature

In [131]:
best_trainer = TwoTowerLitModule.load_from_checkpoint(
    checkpoint_path="/home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/005-two-tower-128-128-neg-4/checkpoints/best-checkpoint-v2.ckpt",
    model = init_model(n_users, n_items, 128,128, args.dropout),
)

best_model = best_trainer.model.cpu()

2025-05-13 16:39:52.188 | INFO     | src.algo.two_tower.trainer:__init__:59 - IDM is not provided. Skipping Evidently ranking metrics logging.


In [132]:
inferrer = TwoTowerInferenceWrapper(best_model)

In [133]:
sample_user_indices = train_df[args.user_col].sample(1).values[0]
sample_item_indices = train_df[args.item_col].sample(1).values[0]

logger.info(f"Sample user: {sample_user_indices}, Sample item: {sample_item_indices}")
sample_input = {"user_id": [sample_user_indices], "item_id": [sample_item_indices]}
sample_output = inferrer.infer([0], [1])
sample_output

2025-05-13 16:39:53.873 | INFO     | __main__:<module>:4 - Sample user: AGQ5ERLI2VUZVYLQV5WYJ5TLGVYA, Sample item: B0C2P7CNWG


array([0.09537707], dtype=float32)

In [134]:
signature = infer_signature(sample_input,sample_output,
)

In [143]:
with mlflow.start_run(run_id=run_id):
    mlflow.pyfunc.log_model(
        python_model=inferrer,
        artifact_path="inferrer",
        artifacts={"idm": mlflow.get_artifact_uri("idm_16407u.json")},
        signature=signature,
        input_example=sample_input,
        registered_model_name="two-tower"
    )

2025/05/13 16:48:02 INFO mlflow.pyfunc: Validating input example against model signature


2025/05/13 16:48:06 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Successfully registered model 'two-tower'.
2025/05/13 16:48:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: two-tower, version 1
Created version '1' of model 'two-tower'.


🏃 View run 005-two-tower-128-128-neg-4 at: http://138.2.61.6:5002/#/experiments/2/runs/fd51bfab00c54c5290ce3691d6aafbbd
🧪 View experiment at: http://138.2.61.6:5002/#/experiments/2


In [145]:
model_version = (
            mlf_client.get_registered_model("two-tower")
            .latest_versions[0]
            .version
        )

In [146]:
mlf_client.set_registered_model_alias(
            name="two-tower", alias="champion", version=model_version
        )

In [147]:
mlf_client.set_model_version_tag(
            name="two-tower",
            version=model_version,
            key="author",
            value="dinhln",
        )